<a href="https://colab.research.google.com/github/waterbean0403/TIL/blob/main/05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""# 교차 검증과 그리드 서치"""

"""## 검증 세트"""

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split


train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target을 만듦
# 여기에서도 test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input()으로 만듦

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

# 원래 5,197개였던 훈련 세트가 4,157개로 줄고, 검증 세트는 1,040개가 됨

(4157, 3) (1040, 3)


In [6]:
# sub_input, sub_target과 val_input, val_target을 사용해 모델을 만들고 평가

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 이 모델은 훈련 세트에 과대적합되어 있음

0.9971133028626413
0.864423076923077


In [7]:
"""## 교차 검증"""

from sklearn.model_selection import cross_validate     # 평가할 모델 객체를 첫 번째 매개변수로 전달하고, 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달

scores = cross_validate(dt, train_input, train_target)
print(scores)

# 이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리 반환
# 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간 의미. 각 키마다 5개의 숫자가 담겨 있음. cross_validate() 함수는 기본적으로 5-폴드 교차 검증 수행
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음

{'fit_time': array([0.00911522, 0.0082655 , 0.00847006, 0.0083251 , 0.00788164]), 'score_time': array([0.00170279, 0.0012033 , 0.0011642 , 0.00112963, 0.00110722]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold     # cross_validate() 함수는 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하는 예시

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
"""## 하이퍼파라미터 튜닝"""

from sklearn.model_selection import GridSearchCV     # GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하므로 별도로 cross_validate() 함수를 호출할 필요 없음

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}     # 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦. 여기서는 0.0001부터 0.0005까지 5개의 값 시도

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)     # GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦

In [14]:
gs.fit(train_input, train_target)     # 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행

dt = gs.best_estimator_     # 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)     # 그리드 서치로 찾은 최적의 매개변수

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])     # 각 매개변수에서 수행한 교차 검증의 평균 점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [18]:
# 결정 트리에서 min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정. 여기에다가 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수를 고름

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(5, 20, 1), 'min_samples_split': range(2, 100, 10)}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

# GridSearchCV 클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있음

0.8683865773302731


In [22]:
"""### 랜덤 서치"""

from scipy.stats import uniform, randint     # 싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음

rgen = randint(0, 10)
rgen.rvs(10)

array([2, 9, 6, 5, 7, 1, 6, 2, 7, 1])

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([119,  90, 101, 108, 109, 102,  91,  93, 103,  84]))

In [24]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.32431559, 0.25103528, 0.01276434, 0.90535439, 0.89428788,
       0.26624686, 0.88183633, 0.40836346, 0.78034102, 0.09041587])

In [25]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(20, 50), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1, 25)}

In [26]:
from sklearn.model_selection import RandomizedSearchCV     # 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 샘플링 횟수를 지정함

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [29]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
